In [10]:
from config import config, db_engine
from etl import *
import psycopg2
from sqlalchemy import text

from gensim.utils import simple_preprocess, tokenize
import gensim.downloader

In [20]:
# Read in id and notes of datasets that have structured data files
query = """
    SELECT
        id, title, notes_en
    FROM datasets
    WHERE id in (SELECT dataset_id FROM resources WHERE file_format in ('CSV', 'JSON')) AND notes_en IS NOT NULL;
"""

df = pd.DataFrame(db_engine().connect().execute(text(query)))

In [14]:
# Inspect df
df.head()

,id,title,notes_en
0,abc37c0b-9dc2-42f3-945c-54be2d1b0f0d,Historical Insolvency Statistics - Receivershi...,Historical monthly insolvencies by receiverships
1,eaee474c-4b7f-4940-8d48-59123d0da436,Historical Insolvency Statistics - North Ameri...,Historical monthly insolvencies by NAICS
2,0968ddc5-710e-4388-b379-184764df6f4c,Darlington New Nuclear Project - Information a...,"This dataset includes links, documents, docume..."
3,e0d1f0bf-699f-437e-805d-4aaa428f96b8,5th National Action Plan on Open Government Re...,This dataset consolidates quarterly progress f...
4,e8402029-2543-4300-bf6a-81a788a08f70,Conditions,The Canada Energy Regulator produces a table s...


In [8]:
def clean_and_preprocess_notes(notes):
    notes = notes.lower() # lowercase all the text
    notes = re.sub(r"\n", "", notes) # Remove new line escape character
    notes = re.sub(r"\r", "", notes) # Remove hard return escape character
    notes = re.sub("\[.+\]\(.+\)", "", notes) # Remove links in markdown language
    notes = re.sub("\*\*", "", notes) # Removing markdown syntax for bolded text

    return notes

def sentence2vec(text, embedding):
    """
    Embed a sentence by averaging the word vectors of the tokenized text. Out-of-vocabulary words are replaced by the zero-vector.
    -----

    Input: text (string)
    Output: embedding vector (np.array)
    """
    model = embedding
    tokenized = simple_preprocess(text)

    word_embeddings = [np.zeros(300)]
    for word in tokenized:
        # if the word is in the model then embed
        if word in model:
            vector = model[word]
        # add zeros for out-of-vocab words
        else:
            vector = np.zeros(300)

        word_embeddings.append(vector)

    # average the word vectors
    sentence_embedding = np.stack(word_embeddings).mean(axis=0)

    return sentence_embedding

def categorize_dataset(text, domain_embeddings: list, vectors, sim_threshold: float = 0.95):
    # For each dataset, get the embedding and get cosine similarity between this vector and domain vector
    # If cosine similarity is below certain threshold, classify it as "Other"
    domain_names = domain_df['domain_name'].tolist()

    # Get embedding for text
    embedding = sentence2vec(text=text, embedding=vectors)

    # Calculate cosine similarities between dataset description and domain description
    cosine_similarities = vectors.cosine_similarities(embedding, domain_embeddings)

    # Check if cosine similarity is above a threshold. If above, then categorize dataset with the highest similarity.
    domain_or_none = np.argmax([np.max(cosine_similarities), sim_threshold])

    if domain_or_none == 0:
        category_id = np.argmax(cosine_similarities)
        category_name = domain_names[category_id]

    else:
        category_name = "No Category"

    return category_name

clean_and_preprocess_notes_lambda = lambda x: clean_and_preprocess_notes(x)

In [11]:
# Load pre-trained word2vec model
model = gensim.downloader.load('fasttext-wiki-news-subwords-300')

INFO:gensim.models.keyedvectors:loading projection weights from C:\Users\andre/gensim-data\fasttext-wiki-news-subwords-300\fasttext-wiki-news-subwords-300.gz
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (999999, 300) matrix of type float32 from C:\\Users\\andre/gensim-data\\fasttext-wiki-news-subwords-300\\fasttext-wiki-news-subwords-300.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2023-11-06T19:02:43.261899', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'load_word2vec_format'}


In [15]:
# Create table for domains
header = ["domain_name", "domain_description"]
table_data = [
    ["Natural Environment", "Marine ecosystems encompass coastal, inland lake, river, and stream environments, supporting diverse biodiversity. Terrestrial ecosystems, such as forests, agroforestry, and grasslands, also contribute to biodiversity. Natural resource conservation and protected areas are crucial for countering natural area loss. Soil conditions directly impact productivity, while wetlands face their own unique conservation challenge"],
    ["BEPINS", "Efficient food resource management is essential for food security, covering cropland, livestock, agriculture, aquaculture, and fisheries. Water resource management focuses on aquifers, freshwater withdrawals, and reducing irrigation runoff. Effective waste and sanitation resource management is vital for public health and the environment. Sustainable land use and urban planning aim to minimize developed area loss and promote green spaces. Resilient power systems ensure energy reliability, balancing renewable and non-renewable sources, while addressing energy intensity. Reliable communications infrastructure and internet connectivity are essential for connectivity. Housing quality, building codes, and addressing vacant structures impact structure vulnerability. Adequate transportation and utility infrastructure are crucial for urban development and mobility."],
    ["Society and Governance", "Community preparedness and resilience are essential for handling disasters. Personal preparedness includes safeguarding property and having insurance coverage, while a robust social safety net and access to support and services are vital. Emergency and civil services play a crucial role in crisis response. Economic stability, employment opportunities, and resilience are key factors, influenced by census statistics and employment rates. Health system adaptation and demographic health statistics impact resilience, including life expectancy, mortality, birth rates, and healthcare capacity. Living standards and human migration are influenced by these factors, affecting vulnerable populations and social demographics, including education, literacy, and gender equality."],
    ["Climate Risk and Systems", "Effective disaster and climate risk management involves understanding exposure and probability to various hazards, including earthquakes, extreme temperatures, floods, storms, and environmental concerns like CO2 emissions and noise pollution. Climate services are crucial for disaster mitigation and early warning systems, enhancing climate resiliency to mitigate the impacts of these hazards."]
]

# Create table
domain_df = pd.DataFrame(table_data, columns=header)

# Get embeddings for domain description
domain_df['embeddings'] = domain_df['domain_description'].apply(lambda x: sentence2vec(text=x, embedding=model))

# Get list of embeddings and names
domain_embeddings = domain_df['embeddings'].tolist()
domain_names = domain_df['domain_name'].tolist()

In [22]:
# clean and preproces notes 
df['notes_en'] = df["notes_en"].apply(lambda x: clean_and_preprocess_notes(x))

# Categorize datasets using cosine similarities
df['domain'] = df['notes_en'].apply(lambda x: categorize_dataset(
    text=x,
    domain_embeddings=domain_embeddings,
    vectors=model,
    sim_threshold = 0.94))

In [24]:
# Check for datasets that have been categorized
df[df['domain'] != "No Category"]

,id,title,notes_en,domain
28,2c09d2fd-9a8e-4d8c-b5af-95747e36eaac,Fisheries Act Registry,the 2019 amended fisheries act (fa) contains a...,BEPINS
71,1b30f3bd-0ad0-4128-916b-66c6dd91dea4,Old Growth Management Areas - Legal - Current,this 'current' spatial data layer is publicly ...,BEPINS
99,f063bff2-d8dd-4cc3-b3a4-00165aba58e1,Old Growth Management Areas - Non Legal - Current,this 'current' spatial data layer is publicly ...,BEPINS
107,2e469ff2-dadb-45ea-af9d-f5683a4b9465,Mental Health and Substance Use Health Services,the healthlink bc mental health and substance ...,Society and Governance
108,88e982b2-0799-4e17-b000-9961ea05b0f0,Emergency Rooms in BC,hospitals and other health care facilities in ...,Society and Governance
...,...,...,...,...
12577,90e4b191-59b3-4ea8-aec4-4e6a7d237673,FluWatch 2014-2015,fluwatch is canada's national surveillance sys...,Society and Governance
12578,5faff0d1-f89d-7391-a306-2d0b26096282,Historical Ambient Air Quality Raw - Estevan,the ministry of environment operates a network...,BEPINS
12581,263db220-a0ec-65e7-041c-8e2163132292,Historical Ambient Air Quality Raw - Regina,the ministry of environment operates a network...,BEPINS
12586,4315264c-a604-432b-b2a8-e47f0608e3b1,FluWatch 2016-2017,fluwatch is canada's national surveillance sys...,Society and Governance
